In [3]:
#  load the dataset Online retail.xlsx

import pandas as pd

# Assuming the file is in your Google Drive, adjust the path if needed
df = pd.read_excel('Online retail.xlsx')
df
# Display the first few rows of the DataFrame
print(df.head())


  shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0                             burgers,meatballs,eggs                                                                                                                                                                             
1                                            chutney                                                                                                                                                                             
2                                     turkey,avocado                                                                                                                                                                             
3  mineral water,milk,energy bar,whole wheat rice...                                            

In [13]:
#  Data Preprocessing:
# Pre-process the dataset to ensure it is suitable for Association rules, this may include handling missing values, removing duplicates, and converting the data to appropriate format.  


processed_df = df.copy() 

# Drop rows with missing values
processed_df.dropna(inplace=True)

# Remove duplicate rows
processed_df.drop_duplicates(inplace=True)

invoice_column = None  # Initialize a variable to store the invoice column name
for col in ['InvoiceNo', 'invoiceno', 'Invoice Number']:
    if col in processed_df.columns:
        invoice_column = col
        break  # Exit the loop if the column is found

if invoice_column is not None:
    # Rename the column to 'InvoiceNo' if necessary
    if invoice_column != 'InvoiceNo':
        processed_df.rename(columns={invoice_column: 'InvoiceNo'}, inplace=True)
        print(f"Column name '{invoice_column}' found and renamed to 'InvoiceNo'")
    
    # Convert the 'InvoiceNo' column to string type
    processed_df['InvoiceNo'] = processed_df['InvoiceNo'].astype(str)

    # Remove transactions with cancelled invoices
    processed_df = processed_df[~processed_df['InvoiceNo'].str.contains('C')]
else:
    print("Error: No suitable invoice column found. Check the column names in your Excel file.")


Error: No suitable invoice column found. Check the column names in your Excel file.


In [15]:
#  load the dataset Online retail.xlsx

import pandas as pd

# Assuming the file is in your Google Drive, adjust the path if needed
df = pd.read_excel('Online retail.xlsx')

# Display the first few rows of the DataFrame
print(df.head())

# prompt: Data Preprocessing:
# Pre-process the dataset to ensure it is suitable for Association rules, this may include handling missing values, removing duplicates, and converting the data to appropriate format.  

# Make a copy of the original DataFrame to avoid modifying it directly
processed_df = df.copy() 

# Check if 'Country' column exists, if not, skip dropping NA
if 'Country' in processed_df.columns:
    # Drop rows with missing values in 'Country' column
    processed_df.dropna(subset=['Country'], inplace=True)
else:
    print("Warning: 'Country' column not found. Skipping missing value removal for this column.")

# Remove duplicate rows
processed_df.drop_duplicates(inplace=True)

# Convert the InvoiceNo to string type
# Check if 'InvoiceNo' exists in the columns, if not, try variations
invoice_column = None  # Initialize a variable to store the invoice column name
for col in ['InvoiceNo', 'invoiceno', 'Invoice Number']:
    if col in processed_df.columns:
        invoice_column = col
        break  # Exit the loop if the column is found

if invoice_column is not None:
    # Rename the column to 'InvoiceNo' if necessary
    if invoice_column != 'InvoiceNo':
        processed_df.rename(columns={invoice_column: 'InvoiceNo'}, inplace=True)
        print(f"Column name '{invoice_column}' found and renamed to 'InvoiceNo'")
    
    # Convert the 'InvoiceNo' column to string type
    processed_df['InvoiceNo'] = processed_df['InvoiceNo'].astype(str)

    # Remove transactions with cancelled invoices
    processed_df = processed_df[~processed_df['InvoiceNo'].str.contains('C')]
else:
    print("Error: No suitable invoice column found. Check the column names in your Excel file.")



  shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0                             burgers,meatballs,eggs                                                                                                                                                                             
1                                            chutney                                                                                                                                                                             
2                                     turkey,avocado                                                                                                                                                                             
3  mineral water,milk,energy bar,whole wheat rice...                                            

In [17]:
#  Analysis and Interpretation:
# •	Analyse the generated rules to identify interesting patterns and relationships between the products.
# •	Interpret the results and provide insights into customer purchasing behaviour based on the discovered rules.

# Import necessary libraries for association rule mining
!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Assuming processed_df is your preprocessed DataFrame
# Create a one-hot encoded DataFrame for association rule mining

# Check if 'InvoiceNo' column exists in processed_df
if 'InvoiceNo' in processed_df.columns:
    basket = processed_df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
    basket[basket > 0] = 1  # Convert quantities to 1 for presence/absence

    # Generate frequent itemsets using apriori
    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

    # Generate association rules
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)


else:
    print("Error: 'InvoiceNo' column not found in processed_df. Please check your preprocessing steps.")

In [19]:
# Interview Questions:
# 1.	What is lift and why is it important in Association rules?
# 2.	What is support and Confidence. How do you calculate them?
# 3.	What are some limitations or challenges of Association rules mining?

# 1. What is lift and why is it important in Association rules?

# Lift measures the ratio of the observed support of a rule to the expected support if the antecedent and consequent were independent.  
# Lift > 1 implies that the items are more likely to be bought together than if they were independent. 
# Lift < 1 implies that the items are less likely to be bought together than if they were independent.
# Lift = 1 indicates that the items are independent.
#
# Importance:  Lift helps to identify the truly interesting rules, filtering out those that might occur simply due to high support for individual items.  A high lift value suggests a strong association between the antecedent and the consequent that is not explainable by chance alone.  It's crucial for focusing on the most impactful relationships.


# 2. What is support and Confidence. How do you calculate them?

# Support: The support of an itemset (or rule) is the proportion of transactions in the dataset that contain that itemset.
#    Calculation:  (Number of transactions containing the itemset) / (Total number of transactions)
#
# Confidence: The confidence of a rule (X => Y) is the conditional probability that a transaction containing X will also contain Y.
#    Calculation: (Support of the itemset X ∪ Y) / (Support of the itemset X)
#
# Example:
# Let's say we have 1000 transactions, and in 100 of those transactions, both item A and item B are present.
#
# Support(A ∪ B) = 100/1000 = 0.1
#
# Now, suppose that item A appears in 200 transactions.
#
# Support(A) = 200/1000 = 0.2
#
# Then the confidence of the rule A => B is:
# Confidence(A => B) = Support(A ∪ B) / Support(A) = 0.1 / 0.2 = 0.5


# 3. What are some limitations or challenges of Association rules mining?

#  Data sparsity: Large datasets often have sparse itemsets, leading to infrequent item combinations. This results in a vast search space and can be computationally expensive.
#
#  High dimensionality: Datasets with many items can lead to a very high number of possible rules. This can overwhelm the system with irrelevant rules, increasing computation time and making the results difficult to analyze.
#
#  Minimum support and confidence thresholds: Setting appropriate thresholds for support and confidence is crucial for identifying meaningful rules. An inappropriate choice of thresholds might result in missing important patterns or discovering noisy and irrelevant relationships.
#
#  Scalability issues: Association rule mining can be computationally intensive, particularly for large datasets. This limits its scalability and the size of problems it can handle effectively.
#
#  Handling categorical and numerical data: Association rule mining is typically designed to handle categorical data. Special pre-processing is needed for numerical data before applying the algorithms.  Binning or other techniques need to be applied carefully.
#
#  Lack of interpretability:  While lift and confidence give information, it's still difficult to directly relate the discovered rules to specific causal relationships.
#
#  Inherent biases:  The data may contain inherent biases which may influence the identified patterns.
